# Introducing `sb4dfritzlib`

I invested some time to understand how to communicate with AVM FRITZ!Box routers using the [official APIs](https://fritz.com/service/schnittstellen/). As a result, I have written my own library `sb4dfritzlib` that will eventually remove the dependency of `sb4dfritz` from the `fritzconnection` library. Not that there's anything wrong with the latter, I just wanted to know how to do it myself so that I know exactly what's going on.

Another motivating factor was the faint hope to get more accurate "real time" power measurements from energy sensors, or at least a better understanding what causes the limitations that I encountered. Recall that `fritzconnection` returns timestamps with second precision, leaving it unclear whether they were obtained by truncating or rounding. While I had little doubt that AVM is the root of this imprecision, I couldn't rule out the data processing `fritzconnection` as a cause. I can now definitively say that `fritzconnection` simply converts the XML data returned by the FRITZ!Box to appropriate Python data types. 

Anyway, here's a summary of what I did:
* I implemented the [SOAP](https://en.wikipedia.org/wiki/SOAP)-based TR-064 interface (see `sb4dfritzlib.connection`).
* I implemented the simpler AHA-HTTP interface and the required login procedure (see `sb4dfritzlib.connection`).
* I implemented various utilities for data conversion (see `sb4dfritzlib.utilities`).
* The core functionality is implemented in `sb4dfritzlib.homeauto` which provides classes `HomeAutoSystem` and `HomeAutoDevice` for object-based interaction with home automation devices.

## A quick demonstration of `sb4dfritzlib`



Let me give you a quick demonstration of `sb4dfritzlib`. The first step is to make a few import and to load the login data from a safe location.

In [ ]:
import sb4dfritzlib
from sb4dfritzlib.connection import FritzUser
from sb4dfritzlib.homeauto import HomeAutoSystem
import json 

CONFIG_FILE = "..\\..\\_private_files\\sb4dfritz_secrets.ini"
with open(CONFIG_FILE,"r") as file:
    CONFIG = json.load(file)
# extract login data
USER = CONFIG['login']['user']
PWD = CONFIG['login']['pwd']
IP = CONFIG['login']['ip']


I'm still looking for a better solution to safely store the login data, but for now this crutch works. If you want to try this yourself, you can enter your data manually or create your own config file. (The last three lines should give you an idea of its structure. It's a JSON file containing a nested dictionary.)

Next, here's how to connect to the home automation system. All that's required is the login data. The smart home devices are detected automatically.

In [3]:
# Create FritzUser from credentials
sb4d_user = FritzUser(USER, PWD, IP)
# Connect to home automation system
sb4d_home_auto_system = HomeAutoSystem(sb4d_user)
# List registered devices
print("Registered home automation devices:")
for device in sb4d_home_auto_system.devices:
    print(" -", device)

Registered home automation devices:
 - l'angolo del caffè (AVM FRITZ!Smart Energy 200)
 - radiatore nel soggiorno (AVM FRITZ!Smart Thermo 301)
 - radiatore musicale (AVM FRITZ!Smart Thermo 301)
 - radiatore nella cucina (AVM FRITZ!Smart Thermo 301)
 - TV etc (AVM FRITZ!Smart Energy 200)


Under the hood, the instantiation of `HomeAutoSystem` first log in to obtains a valied session id, then asks for a list of connected devices, and creates a corresponding instance of the `HomeAutoDevice` class defined in the `sb4dfritzlib.homeauto` sub-package.

In my case, the first and last devices are smart plugs which function as on/off switch for my espresso machine and TV unit. Let's give them better names check the current power consumption of my coffee machine:

In [4]:
# Give coffee machine and tv better names
sb4d_caffe = sb4d_home_auto_system.devices[0]
sb4d_tv = sb4d_home_auto_system.devices[4]

In [5]:
# Get latest power measurement of cofee machine with timing information
power_record = sb4d_caffe.get_latest_power_record()
print("Current power consumption and timing information:")
for key, val in power_record.items():
    print(f" - {key:15s} {val}")

Current power consumption and timing information:
 - power           0.0
 - datatime        2025-08-21 12:45:28
 - starttime       2025-08-21 12:46:00.491786
 - endtime         2025-08-21 12:46:01.304348
 - duration        0.812562
 - latency         33.304348
 - offset          -32.491786


As you can guess, the smart plug reported 0.0 W as `power` at the time `datatime`. 
The parameters `starttime` and `endtime` record the times that the network request to obtain the data was sent and received, respectively.
Lastly, `duration` is the total request time in seconds while `latency` and `offset` compare the `endtime` and `starttime` to `datatime`, respectively.
The large values are typical if no request has been made recently. 

At this point, it should not go unnotices that a latency of over 50 seconds, meaning that the measurement was over 50 seconds ago, can hardly be considered "real time". In fact, lactencies of ~120 seconds are not uncommon on such "cold requests" after longer time without request.

Let's make a few more requests to see what happens over time.

In [6]:
OUTPUT_TEMPLATE = """{:4s} Request: {:s} | Power: {:6.1f} W | Timestamp: {:s} | Duration: {:0.2f} | Latency: {:6.2f}"""

In [7]:
for k in range(25):
    data = sb4d_caffe.get_latest_power_record()
    output = OUTPUT_TEMPLATE.format(
        f"({k+1})",
        data['starttime'].strftime("%H:%M:%S.%f"),
        data['power'],
        data['datatime'].strftime("%H:%M:%S"),
        data['duration'],
        data['latency'],
    )
    print(output)


(1)  Request: 12:47:47.221628 | Power:    0.0 W | Timestamp: 12:47:40 | Duration: 0.88 | Latency:   8.10
(2)  Request: 12:47:48.098977 | Power:    0.0 W | Timestamp: 12:47:40 | Duration: 0.83 | Latency:   8.93
(3)  Request: 12:47:48.931169 | Power:    0.0 W | Timestamp: 12:47:40 | Duration: 0.80 | Latency:   9.73
(4)  Request: 12:47:49.732113 | Power:    0.0 W | Timestamp: 12:47:40 | Duration: 0.84 | Latency:  10.57
(5)  Request: 12:47:50.570065 | Power:    0.0 W | Timestamp: 12:47:40 | Duration: 0.83 | Latency:  11.40
(6)  Request: 12:47:51.403961 | Power:    0.0 W | Timestamp: 12:47:50 | Duration: 0.84 | Latency:   2.24
(7)  Request: 12:47:52.241256 | Power:    0.0 W | Timestamp: 12:47:50 | Duration: 0.86 | Latency:   3.10
(8)  Request: 12:47:53.099781 | Power:    0.0 W | Timestamp: 12:47:50 | Duration: 0.83 | Latency:   3.93
(9)  Request: 12:47:53.928298 | Power:    0.0 W | Timestamp: 12:47:50 | Duration: 0.80 | Latency:   4.73
(10) Request: 12:47:54.729573 | Power:    0.0 W | Times

From the above results, we can take away a few thing:

* Request normally take around 0.8 seconds, occasionally a little longer.
* The timestamp jumps 10 second intervals, which indicates that power is measured every 10 seconds. Somewhat surprisingly, this means that **data requests *do not* trigger a measurements!** Measurements are taken periodically at times unrelated to the requests.
* On repeated requests, the latency does not exceed 10 seconds by much. This indicates some undocumented "sleep mode" which reduces the update frequency in longer time stretches without data requests.

If you're interested in real time information, this is bad news. A random request will give you a power value recorded some time in the last ~12.5 seconds, if you're lucky. Again, I would not consider this as "real time". After a little thought, you should realize that,  given the network latency (reflected in the request duration) and the second presicion in the timestamp, it's impossible to determine the time of measurement precisely from the provided data. There is always going to be some latency, and in bad cases you cannot expect to do better than ~2 seconds of latency. 

My personal motivation was to switch off my coffee machine only when it's idle (indicated by a power consumption below 5 W). The problem is that the machine has irregular heating cycles, even if it's not actively being used. Initially, I thought that this was easy: just ask for the current power consumption, and if it is below a given threshold, switch off the smart plug. Clearly, this doesn't work because of the described "real time" issues. 

So, what to do about this?

## Problem: How to get device statistics with low latency?

Well, I tried to be smart about it and set up a parallel request cycle which synchronizes itself with the measurement cycle. I tried various algorithms, but they all took rather long and turned out to be error prone. My current algorithm is rather simple:


* Keep sending repeated power data requests as above in a `while` loop.
* Record only the measurements where the timestamp jumps.
* Declare what it means to be "idle" by setting thresholds for power consumption and request duration as well as a number of required idle measurements.
* Check the last recorded measurements if the criteria are satisfied. If so, send the switch off command.

Although this is not exactly pretty, it gets the job done better than all other ideas I've tried. It's fast and the latency is not too far from optimal. While I am a little worried about the amount of data requests (roughly one every 0.8 seconds!), so far the algorithm has been stable.

The algorithm is implemented in a method `.switch_off_when_idle()` for the `HomeAutoDevice` class.

